# 法人インフォメーションをD3.jsで利用する

まずはデフォルトでどのぐらいの長さのオブジェクトを取得できるか試してみます

In [5]:
import py_d3
py_d3.load_ipython_extension(get_ipython())

In [2]:
%%d3
<div id="test"></div>
<script>
var q = encodeURIComponent(get_sparql()),
    endpoint = "http://api.hojin-info.go.jp/sparql";

function get_sparql(){
    return [
        "PREFIX hj: <http://hojin-info.go.jp/ns/domain/biz/1#>",
        "PREFIX ic: <http://imi.go.jp/ns/core/rdf#>",
        "SELECT ?v ?name FROM <http://hojin-info.go.jp/graph/hojin>",
        "WHERE {",
        "?d hj:法人基本情報 ?db .",
        "?db ic:ID/ic:識別値 ?v ;",
        "ic:名称/ic:表記 ?name .",
        "}"
    ].join(" ");
}

d3.json(endpoint + "?query=" + q , function(error, data){
    if(error){
        console.log(error)
    }else{
        show(data)
    }
});

function show(data){
    items = data['results']['bindings']
    d3.select('#test').append("text").text(items.length)
}

</script>

javascriptのSPARQLの記述、テンプレートリテラルを使うともっと簡単になるとのこと。下記のように記述できます。

In [10]:
%%d3
<div id="test"></div>
<script>
var q = encodeURIComponent(get_sparql()),
    endpoint = "http://api.hojin-info.go.jp/sparql";

function get_sparql(){
    return `
        PREFIX hj: <http://hojin-info.go.jp/ns/domain/biz/1#> 
        PREFIX ic: <http://imi.go.jp/ns/core/rdf#> 
        SELECT ?v ?name FROM <http://hojin-info.go.jp/graph/hojin> 
        WHERE {
        ?d hj:法人基本情報 ?db . 
        ?db ic:ID/ic:識別値 ?v ; 
        ic:名称/ic:表記 ?name . 
        }`
}

d3.json(endpoint + "?query=" + q , function(error, data){
    if(error){
        console.log(error)
    }else{
        show(data)
    }
});

function show(data){
    items = data['results']['bindings']
    d3.select('#test').append("text").text(items.length)
}

</script>

次に、東京都の法人名を取得してみます

In [75]:
%%d3
<div id="test"></div>
<script>
var q = encodeURIComponent(get_sparql()),
    endpoint = "http://api.hojin-info.go.jp/sparql";

function get_sparql(){
    return `
        PREFIX hj: <http://hojin-info.go.jp/ns/domain/biz/1#> 
        PREFIX ic: <http://imi.go.jp/ns/core/rdf#> 
        SELECT ?v ?name FROM <http://hojin-info.go.jp/graph/hojin> 
        WHERE {
        ?d hj:法人基本情報 ?db . 
        ?db ic:ID/ic:識別値 ?v ; 
        ic:名称/ic:表記 ?name ; 
        ic:住所/ic:都道府県 '東京都' .
        } LIMIT 10`
}

d3.json(endpoint + "?query=" + q , function(error, data){
         if(error){
        console.log(error)
    }else{
        show(data)
    }
});

function show(data){
    items = data['results']['bindings']
    names = items.map(function(d){return d.name})
    d3.select('#test').selectAll("tspan")
        .data(names).enter()
        .append("tspan")
        .text(function(d){return d.value + ", " })
}

</script>